In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import re
 
from torch.utils.data import DataLoader, Dataset 
from transformers import AutoTokenizer 
from sklearn.model_selection import train_test_split
import random
import numpy as np
import torch 
import os 
from tqdm import tqdm

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)

    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.use_deterministic_algorithms(True)

    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'

set_seed(42)

g:\vseross\one-step_E-task\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def hit_at_k(y_true, y_pred, k=3):
    hits = 0
    total = len(y_true)
    
    for true_labels, pred_labels in zip(y_true, y_pred):
        if isinstance(true_labels, str):
            true_set = set(true_labels.split('|'))
        else:
            true_set = set(true_labels)
        
        pred_top_k = pred_labels[:k] if len(pred_labels) >= k else pred_labels
        
        if any(label in true_set for label in pred_top_k):
            hits += 1
    
    return hits / total if total > 0 else 0.0

def top_k_to_binary(probas, k=3):
    probas = np.array(probas)
    result = np.zeros_like(probas, dtype=int)
    top_k_indices = np.argsort(probas)[-k:][::-1]
    result[top_k_indices] = 1
    return result

In [3]:
from sklearn.preprocessing import MultiLabelBinarizer

data = pd.read_csv('train.csv')
data['labels_new'] = data['labels_str'].apply(lambda x: x.split('|'))

mlb = MultiLabelBinarizer()
mlb.fit(data['labels_new'].values)

MultiLabelBinarizer()

In [4]:
class MyDataset(Dataset):
    def __init__(self, data: str, MLB: MultiLabelBinarizer):
        self.data = data 

        if self.data == 'train' or self.data == 'valid':
            self.df = pd.read_csv('train.csv')
            self.df['labels_new'] = self.df['labels_str'].apply(lambda x: x.split('|'))
            self.df['full_description_len'] = self.df['full_description'].apply(lambda x: len(x)) 
            train_df, valid_df = train_test_split(self.df, test_size=1000, random_state=42)
            
            self.df = train_df if self.data == 'train' else valid_df
        else:
            self.df = pd.read_csv('test.csv')

        def clean_text(text):
            text = str(text).lower()
            text = re.sub(r'[^\w\s]', ' ', text)
            text = re.sub(r'\s+', ' ', text)
            return text.strip()

        self.df['full_text'] = (
            self.df['app_name'].fillna('') + ' ' + 
            self.df['shortDescription'].fillna('') + ' ' + 
            self.df['full_description'].fillna('')
        )
        self.df['full_text'] = self.df['full_text'].apply(clean_text)
        
        self.MLB = MLB 

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        text = row['full_text']

        if self.data != 'test': 
            label = row['labels_new']
            label = mlb.transform([label])[0] 
            return text, torch.tensor(label, dtype=torch.float)
        else:
            return (text, )  

In [5]:
from transformers import AutoTokenizer, AutoModel
import torch.nn as nn 

class CustomBert(nn.Module):
    def __init__(self, num_classes=45):
        super().__init__()
        self.model_bert = AutoModel.from_pretrained(
            "cointegrated/LaBSE-en-ru",
            use_safetensors=True
        )
         #"sentence-transformers/paraphrase-multilingual-mpnet-base-v2", 
        self.fc1 = nn.Linear(768, num_classes)

    def mean_pooling(self, model_output, attention_mask):
        token_embeddings = model_output[0]
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

    def forward(self, x):
        outputs = self.model_bert(**x)
        pooled_output = self.mean_pooling(outputs, x['attention_mask'])
        return self.fc1(pooled_output)

In [6]:
data_train = MyDataset('train', mlb)
data_valid = MyDataset('valid', mlb)
data_test = MyDataset('test', mlb)

data_train = DataLoader(data_train, 16, shuffle=True)
data_valid = DataLoader(data_valid, 16, shuffle=False)
data_test = DataLoader(data_test, 16, shuffle=False)

In [ ]:
import torch.nn.functional as F

device = 'cuda:0'

model = CustomBert()
model.to(device)

tokenizer = AutoTokenizer.from_pretrained("cointegrated/LaBSE-en-ru")  
loss_fn = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5, weight_decay=0.01)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20)

EPOCHS = 20

best_score = 0
early_stopp = 1
now_num = 0 

for i in range(EPOCHS):
    print('='*50)
    print(f"EPOCH: {i+1}")
    print('='*50)
    
    train_sum = 0
    
    model.train()

    for text_batch, label_batch in tqdm(data_train):
        text_batch = tokenizer(text_batch, return_tensors='pt', padding=True, truncation=True,
                               max_length=256)
        text_batch = text_batch.to(device)
        
        label_batch = label_batch.to(device)

        optimizer.zero_grad()

        outputs = model(text_batch)

        loss = loss_fn(outputs, label_batch)
        loss.backward()
        optimizer.step()

        train_sum += loss.item()

    print(f'Train Loss: {train_sum / len(data_train)}')

    model.eval()

    val_sum = 0
    y_pred_proba = []
    y_true = []

    with torch.no_grad():
        for text_batch, label_batch in tqdm(data_valid):
            text_batch = tokenizer(text_batch, return_tensors='pt', padding=True, 
                                   truncation=True, max_length=256)
            
            text_batch = text_batch.to(device)
            label_batch = label_batch.to(device)
            outputs = model(text_batch)
            val_sum += loss_fn(outputs, label_batch).item()
       
            for x in torch.sigmoid(outputs).cpu().numpy(): 
                y_pred_proba.append(top_k_to_binary(x))

            for x in label_batch.cpu().numpy():
                y_true.append(x)

    y_true = mlb.inverse_transform(np.array(y_true))
    y_pred = mlb.inverse_transform(np.array(y_pred_proba))
    hit_at_2_score = hit_at_k(y_true, y_pred, k=3)
    print(f'Val Loss: {val_sum / len(data_valid)}, Metric: {hit_at_2_score}')

    if hit_at_2_score > best_score:
        best_score = hit_at_2_score 
        torch.save(model, 'best_model.pth')
        now_num = 0 
    else:
        now_num += 1 

        if now_num == early_stopp:
            break
    
    scheduler.step()

In [8]:
model = torch.load('best_model.pth', map_location=device)

C:\Users\djadc\AppData\Local\Temp\ipykernel_23644\1655999588.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load('best_model.pth', map_location=device)


In [9]:
test = pd.read_csv('test.csv')
y_pred_proba = []

with torch.no_grad():
    for text_batch in tqdm(data_test):
        text_batch = tokenizer(text_batch[0], return_tensors='pt', padding=True, 
                               truncation=True, max_length=512)
        text_batch = text_batch.to(device)
        outputs = model(text_batch)
        
        for x in torch.sigmoid(outputs).cpu().numpy():
            y_pred_proba.append(top_k_to_binary(x))

y_pred = mlb.inverse_transform(np.array(y_pred_proba))

submission = pd.DataFrame({
    'app_name': test['app_name'].values,
    'labels_str': ['|'.join(row) for row in y_pred]
})

submission.to_csv('submission.tsv', sep='\t', index=False)
submission

100%|██████████| 941/941 [01:48<00:00,  8.69it/s]


app_name  \
0                                        Lemon clicker   
1                                 Memo Английский язык   
2                                     Slave Man Rescue   
3                                Taking Care of Granny   
4                             Escape From Classic Room   
...                                                ...   
15041                                        X-Scooter   
15042                                         RadMeter   
15043                                   Мой Белтелеком   
15044                          Поиск по MAC адресу PRO   
15045  Язык тела - Обмани меня. Жесты и микровыражения   

                              labels_str  
0                arcade|casual|simulator  
1                  books|education|tools  
2                action|adventure|puzzle  
3                 arcade|casual|children  
4                   arcade|casual|puzzle  
...                                  ...  
15041      entertainment|simulator|tools  
15042      entertainment|lifestyle|tools  
15043      adsAndServices|business|tools  
15044      entertainment|lifestyle|tools  
15045  education|entertainment|lifestyle  

[15046 rows x 2 columns]

In [ ]:
submission

app_name  \
0                                        Lemon clicker   
1                                 Memo Английский язык   
2                                     Slave Man Rescue   
3                                Taking Care of Granny   
4                             Escape From Classic Room   
...                                                ...   
15041                                        X-Scooter   
15042                                         RadMeter   
15043                                   Мой Белтелеком   
15044                          Поиск по MAC адресу PRO   
15045  Язык тела - Обмани меня. Жесты и микровыражения   

                              labels_str  
0                arcade|puzzle|simulator  
1              children|puzzle|simulator  
2             adventure|puzzle|simulator  
3              children|puzzle|simulator  
4                arcade|puzzle|simulator  
...                                  ...  
15041             puzzle|arcade|strategy  
15042      puzzle|strategy|entertainment  
15043          puzzle|strategy|simulator  
15044           puzzle|strategy|business  
15045  entertainment|lifestyle|education  

[15046 rows x 2 columns]